## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-15-32-39 +0000,nyt,"After Renee Good’s Killing, Some Conservatives...",https://www.nytimes.com/2026/01/17/us/politics...
1,2026-01-17-15-28-27 +0000,nyt,Uganda’s President Museveni Is Declared Electi...,https://www.nytimes.com/2026/01/17/world/afric...
2,2026-01-17-15-26-55 +0000,nypost,‘Worst of the worst’ illegal migrants busted b...,https://nypost.com/2026/01/17/us-news/illegal-...
3,2026-01-17-15-11-24 +0000,startribune,"Live: State, Minneapolis officials prepare for...",https://www.startribune.com/ice-raids-minnesot...
4,2026-01-17-15-09-32 +0000,wapo,Minneapolis braces for march called by pardone...,https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,61
17,ice,20
56,iran,18
469,new,13
58,leader,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2026-01-17-13-33-32 +0000,nypost,Iran’s supreme leader blasts Trump as a ‘crimi...,https://nypost.com/2026/01/17/world-news/ayato...,123
212,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,123
168,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...,118
223,2026-01-16-20-23-34 +0000,nypost,Trump admin slams LA Mayor Karen Bass after sh...,https://nypost.com/2026/01/16/us-news/trump-ad...,113
108,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,123,2026-01-17-13-33-32 +0000,nypost,Iran’s supreme leader blasts Trump as a ‘crimi...,https://nypost.com/2026/01/17/world-news/ayato...
137,78,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
212,62,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
168,45,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...
71,44,2026-01-17-10-00-29 +0000,wapo,Thousands prayed for a new Venezuela. One moth...,https://www.washingtonpost.com/world/interacti...
200,39,2026-01-16-21-24-32 +0000,cbc,Here's what you need to know about the Canada-...,https://www.cbc.ca/news/politics/what-is-in-ca...
220,34,2026-01-16-20-34-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
99,34,2026-01-17-05-24-01 +0000,nypost,Feds in Minneapolis immigration operation can’...,https://nypost.com/2026/01/17/us-news/feds-in-...
203,33,2026-01-16-21-19-26 +0000,nypost,Trump and governors unveil plan to fight AI el...,https://nypost.com/2026/01/16/us-news/trump-an...
146,30,2026-01-17-00-22-35 +0000,latimes,'Largest outbreak that we've seen in Californi...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
